# Forecast Net Demand

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv('.env')

# Get the values of host, user, pswd, db, and schema from the environment variables
host = os.getenv('host')
user = os.getenv('user')
pswd = os.getenv('pswd')
db = os.getenv('db')
schema = os.getenv('schema')


# Use the values as needed
engine = create_engine(
    f"postgresql://{user}:{pswd}@{host}/{db}?options=-csearch_path%3D{schema}", echo=False)
conn = engine.connect()

## Import data from CSV to PostgreSQL

This step is used for testing purposes.

Set `IMPORT_DATA` to `False` to skip this step.

In [2]:
IMPORT_DATA = False

In [3]:
import pandas as pd
import datetime as dt

if IMPORT_DATA:
    
    # Load and filer data from csv file
    
    rt_dpr = pd.read_csv('./data/RT_DPR.csv')
    rt_dpr = rt_dpr[['Date', 'Period', 'Demand', 'TCL', 'Transmission_Loss']]
    rt_dpr['Transmission_Loss'] = rt_dpr['Transmission_Loss'].fillna(0)
    rt_dpr = rt_dpr[rt_dpr['Date'] > '2023-06-30']
    rt_dpr = rt_dpr.sort_values(by=['Date', 'Period'])
    rt_dpr.reset_index(drop=True, inplace=True)
    
    vc_per = pd.read_csv('./data/VCData_Period.csv')
    
    # !!! The Real_Time_DPR table here is different from the one Matthew uses. Don't replace.
    # rt_dpr.to_sql('Real_Time_DPR', conn, if_exists='replace', index=False)
    vc_per.to_sql('VCData_Period', conn, if_exists='replace', index=False)

## Data from DB

In [4]:
import datetime as dt

now = dt.datetime.now()
date = now.strftime("%Y-%m-%d")
time = now.strftime("%H:%M")

period = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1


if period + 1 > 48:
    next_period = 1
    next_date = now + dt.timedelta(days=1)
    next_date = next_date.strftime("%Y-%m-%d")
else:
    next_period = period + 1
    next_date = date

# next_date = '2024-03-25' # A hard-coded value for testing
# next_period = 33 # A hard-coded value for testing
print(f"Now is {date} {time} Period {period}")
print(f"To predict: {next_date} Period {next_period}")

Now is 2024-03-26 16:34 Period 34
To predict: 2024-03-26 Period 35


In [5]:
rt_dpr = pd.read_sql(f"""
                     SELECT "Date", "Period", "Demand", "TCL", "Transmission_Loss"
	                 FROM public."Real_Time_DPR"
                     WHERE ("Date" < '{date}' OR ("Date" = '{date}' AND "Period" < {next_period}))
                     ORDER BY "Date" DESC, "Period" DESC  
                     LIMIT 336
                     """, conn)
rt_dpr.sort_values(by=['Date', 'Period'], inplace=True)
rt_dpr.reset_index(drop=True, inplace=True)
rt_dpr.head(2)

,Date,Period,Demand,TCL,Transmission_Loss
0,2024-03-19,35,7203.81,11.0,37.122
1,2024-03-19,36,7088.09,0.0,36.877


In [6]:
rt_dpr.tail(2)

,Date,Period,Demand,TCL,Transmission_Loss
334,2024-03-26,33,7116.707,0.0,38.499
335,2024-03-26,34,7138.118,0.0,38.902


In [7]:
vc_per = pd.read_sql('SELECT * FROM public."VCData_Period"', conn)
vc_per

,Year,Quarter,Period,TCQ_Weekday,TCQ_Weekend_PH
0,2023,3,1,457184.607704,486146.668221
1,2023,3,2,431743.883051,465108.391557
2,2023,3,3,415236.248998,448268.380324
3,2023,3,4,403405.811743,434353.333280
4,2023,3,5,394503.297641,422426.659436
...,...,...,...,...,...
187,2024,2,44,521795.957581,511745.260075
188,2024,2,45,509037.825497,503120.417756
189,2024,2,46,492624.774907,490075.053877
190,2024,2,47,471344.391703,474155.106133


In [12]:
vc_per[(vc_per['Year'] == 2024) & (
    vc_per['Quarter'] == 1)]['TCQ_Weekday'].values

array([355818.7257016 , 357142.41829544, 341690.20316031, 335844.76203262,
       329430.3107078 , 348038.45509215, 320351.36985643, 357264.25035328,
       350714.82426238, 325854.36562452, 342973.11435316, 334723.85096751,
       410146.20395788, 384326.09265607, 416354.8531033 , 421831.70017212,
       431490.49144217, 441064.89190246, 453713.71262424, 461585.63594439,
       472177.88088323, 476815.73361418, 480049.71048044, 480193.0664062 ,
       476069.10992465, 473187.7455232 , 475633.71332899, 478328.11386127,
       485156.61973021, 484623.77805866, 485591.815491  , 488310.78482161,
       494884.29735853, 499796.9055721 , 505555.91223553, 508903.72230257,
       526114.23661216, 517712.0850365 , 532676.17023474, 523850.65876626,
       514780.43705941, 539775.95044866, 532980.85011206, 522670.88887747,
       492108.97376514, 488650.25360335, 435305.89862802, 394272.55105063])

In [15]:
import holidays

# Calculate required data fields

sg_holidays = holidays.country_holidays('SG')

rt_dpr['Total Demand'] = rt_dpr['Demand'] + rt_dpr['TCL'] + rt_dpr['Transmission_Loss']
view = rt_dpr[['Date', 'Period', 'Total Demand']].copy()

def find_tcq(row):
    # print(row)
    date_obj = dt.datetime.strptime(str(row['Date']), '%Y-%m-%d')
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    
    period = row['Period']
    
    isWeekend = 1 if date_obj.isoweekday() > 5 else 0
    isPublicHoliday = date_obj in sg_holidays
    
    if isWeekend or isPublicHoliday:
        # print(f"Date: {date_obj} isWeekend: {isWeekend} isPublicHoliday: {isPublicHoliday}")
        tcq = vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter) & (vc_per['Period'] == period)]['TCQ_Weekday'].values[0] / 1000
    else:
        tcq = vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter) & (vc_per['Period'] == period)]['TCQ_Weekend_PH'].values[0] / 1000

    # print(f"Date: {date_obj} TCQ: {tcq}")
    return tcq

view['TCQ'] = view.apply(lambda row: find_tcq(row), axis=1)
view['Net Demand'] = view['Total Demand'] - view['TCQ']
view.reset_index(drop=True, inplace=True)
# view.head(2)

In [17]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import os
import glob

# Load the most recent scaler file
resDir = './model'
newestDir = max(glob.glob(os.path.join(resDir, '*/')), key=os.path.getmtime)
newestDir

'./model/20240326_1239/'

In [18]:
scaler_files = glob.glob(os.path.join(newestDir, "*.pkl"))
print("Scaler files:", scaler_files)
scaler = joblib.load(scaler_files[0])
print("Loaded scaler:", scaler_files[0])

# Perform data preprocessing as before
data = view.copy()
data['Target'] = data['Net Demand']
data['Target'] = scaler.fit_transform(data['Target'].values.reshape(-1,1))

# Create dataset for prediction
def create_dataset(dataset):
    return np.array([dataset])

predict_X = create_dataset(data['Target'].values)

# Reshape input to be [samples, time steps, features]
predict_X = np.reshape(predict_X, (predict_X.shape[0], predict_X.shape[1], 1))
print(f"Predict_X shape: {predict_X.shape}")

Scaler files: ['./model/20240326_1239/robustScaler.pkl']
Loaded scaler: ./model/20240326_1239/robustScaler.pkl
Predict_X shape: (1, 336, 1)


## Predict using trained model

In [19]:
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.keras.utils.disable_interactive_logging()

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-03-26 16:36:39.145608: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 16:36:39.145636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 16:36:39.146602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 16:36:39.152213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 16:36:39.740768: W tensorflow/compiler/tf2

Num GPUs Available:  1


2024-03-26 16:36:40.218392: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 16:36:40.254970: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 16:36:40.255232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [20]:
import os
import glob
from keras.models import load_model

# Get a list of all model files in the directory
model_files = glob.glob(os.path.join(newestDir, "*.keras"))

# Sort the list of model files by modification time (most recent first)
model_files.sort(key=os.path.getmtime, reverse=True)

# Select the most recent model file
most_recent_model_file = model_files[0]

# Load the selected model
model = load_model(most_recent_model_file)

# Print the path of the loaded model for verification
print("Loaded model:", most_recent_model_file)


# Make predictions
predict_result = model.predict(predict_X)

# Invert predictions to original scale
inverted_predictions = scaler.inverse_transform(predict_result)


2024-03-26 16:36:41.331654: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 16:36:41.332009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 16:36:41.332170: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loaded model: ./model/20240326_1239/bi-lstm.keras


2024-03-26 16:36:43.113609: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


In [21]:
# Print or use the predictions as needed
print(f"Predictions: {inverted_predictions[0][0]}")

Predictions: 6699.34765625
